Based on TFX Keras Component Tuturial, altered for Sepsis prediction. https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#setup

# Background

This demonstration is based on the TFX Keras Component Tutorial published on https://www.tensorflow.org/tfx/tutorials/tfx/components_keras

This notebook demonstrates how to use TFX in Jupyter on Red Hat OpenShift using Open Data Hub. The ineractive notebook introduces slights differences in an actual pipeline, like orchestration and metadata artifacts.

This notebook demonstrates how to use TFX in a Jupyter on Red Hat OpenShift. We will walk through an interactive notebook and pipeline construction.

# Orchestration

In production, you use an orchestrator like: 
1. Apache Airflow
1. Kubeflow Pipelines
1. Argo

...to orchestrate pre-defined pipeline graph of TFX components.

In a notebook, the notebook is the orchestrator.

# Metadata

In production, you access metadata through an API, like TFX ML Metadata "MLMD" API.

In production, MLMD stores metadata in a database like MySQL or SQLite.

In a notebook, payloads are stored in an ephemeral SQLite database under /tmp on the Jupyter server.

# Setup

First, we install and import the necessary packages, set up paths, and download data.

In [1]:
# quietly upgrade pip
!pip install --upgrade pip -q

# install tfx as user
!pip install -Uq tfx

# quiet install packages 
!pip install -Uq poetry
!poetry -q install

/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


## Restart Kernel
If you are running this the first time that you run the cell above, you must restart the runtime (Kernel > Restart Kernel ...). This is because of the way the notebook loads packages....for now during dev

# Import

We import necessary packages, including standard TFX component classes.

In [2]:
# import packages
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

2022-04-21 17:11:21.337449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-21 17:11:21.337507: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Let's check the library versions.

In [3]:
# check versions installed
# expected TensorFlow version: 2.7.1 TFX version: 1.6.1
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.0
TFX version: 1.7.1


# Set up the pipeline paths

In [4]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Vitals Pipeline example.
_vitals_root = os.path.join(_tfx_root, '../pipeline/sepsis_vitals')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), '../models/sepsis_vitals')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

# Download example data

We download the example dataset for use in our TFX pipeline.

In [5]:
# defines the temporary directory for the sepsis vitals data
_data_root = tempfile.mkdtemp(prefix='sepsis-data')

# this block is for the raw vitals labeled training data
# defines the github path where the data exists, even though it is also cloned in
VITALS_PATH = 'https://raw.githubusercontent.com/redhat-naps-da/mlops-prototype/main/data/dataSepsis/csv_format/pat_vitals_labeled-dataSepsis.csv'
# defines the full local temporary path
_vitals_filepath = os.path.join(_data_root, "pat_vitals_labeled-dataSepsis.csv")
# pulls the data into the newly created temporary filepath
urllib.request.urlretrieve(VITALS_PATH, _vitals_filepath)

# defines the temporary directory for the sepsis vitals data
_attr_root = tempfile.mkdtemp(prefix='sepsis-data')

# this block is for the training data attribute definitions
# defines the github path where the data attributes exists, even though it is also cloned in
ATTR_PATH = 'https://raw.githubusercontent.com/redhat-na-ssa/mlops-prototype/main/data/dataSepsis/csv_format/attribute_definitions.csv'
# defines the full local temporary path
_attr_filepath = os.path.join(_attr_root, "attribute_definitions.csv")
# pulls the data into the newly created temporary filepath
urllib.request.urlretrieve(ATTR_PATH, _attr_filepath)

('/tmp/sepsis-datayx7714m9/attribute_definitions.csv',
 <http.client.HTTPMessage at 0x7f95e9744e80>)

Take a quick look at the CSV file.

In [6]:
# view the vitals attribute data
!head -n 13 {_attr_filepath}

target_file,attribute_name,attribute_definition
,List all the attributes in the dataset. Label continuous attributes with c and discrete with d,
,,
pat_vitals_labeled.csv,Vital signs (columns 1-8),"Doctors order, basis every 4 hours, least invasive, cost least amount of money"
pat_vitals_labeled.csv,HR,Heart rate (beats per minute)
pat_vitals_labeled.csv,O2Sat,Pulse oximetry (%)
pat_vitals_labeled.csv,Temp,Temperature (Deg C)
pat_vitals_labeled.csv,SBP,Systolic BP (mm Hg)
pat_vitals_labeled.csv,MAP,Mean arterial pressure (mm Hg)
pat_vitals_labeled.csv,DBP,Diastolic BP (mm Hg)
pat_vitals_labeled.csv,Resp,Respiration rate (breaths per minute)
pat_vitals_labeled.csv,EtCO2,End tidal carbon dioxide (mm Hg)
pat_vitals_labeled.csv,isSepsis,"For sepsis patients, SepsisLabel is  1 "


In [7]:
# view the first 10 lines of the data
!head {_data_filepath}

head: cannot open '{_data_filepath}' for reading: No such file or directory


# Create the interactive context

Last, we create an InteractiveContext, which will allow us to run TFX components interactively in this notebook.

In [8]:
context = InteractiveContext()

# Run TFX components interactively

In the cells that follow, we create TFX components one-by-one, run each of them, and visualize their output artifacts.

## Data Ingestion with ExampleGen
The ExampleGen component is usually at the start of a TFX pipeline. It will:

- Split data into training and evaluation sets (by default, 2/3 training + 1/3 eval)
- Convert data into the tf.Example format (learn more here)
- Copy data into the _tfx_root directory for other components to access

ExampleGen takes as input the path to your data source. In our case, this is the _data_root path that contains the downloaded CSV.

## Enabling Cache

When using the InteractiveContext in a notebook to develop a pipeline you can control when individual components will cache their outputs. 
1. Set enable_cache to True when you want to reuse the previous output artifacts that the component generated. 
1. Set enable_cache to False when you want to recompute the output artifacts for a component, if you are making changes to the code for example.

In [9]:
# ExampleGen 
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
# enable cache
context.run(example_gen, enable_cache=False)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/sepsis-datahttqzjz6/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Let's examine the output artifacts of ExampleGen. This component produces two artifacts, training examples and evaluation examples:

In [10]:
# Examine ExampleGen artifact
# Expect to see train and eval
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/CsvExampleGen/examples/1


We can also take a look at the first training example:

In [11]:
# Examine train examples
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "DBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "EtCO2"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "HR"
    value {
      int64_list {
        value: 63
      }
    }
  }
  feature {
    key: "MAP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "O2Sat"
    value {
      float_list {
        value: 90.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      int64_list {
        value: 17
      }
    }
  }
  feature {
    key: "SBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 40.29999923706055
      }
    }
  }
  feature {
    key: "isSepsis"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "patient_id"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
 

2022-04-21 17:11:48.588085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-21 17:11:48.588141: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-21 17:11:48.588172: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterhub-nb-dmarcus): /proc/driver/nvidia/version does not exist
2022-04-21 17:11:48.588650: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Analysis with StatisticsGen

The StatisticsGen component computes statistics over your dataset for data analysis, as well as for use in downstream components. It uses the TensorFlow Data Validation library.

StatisticsGen takes as input the dataset we just ingested using ExampleGen.

In [12]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

After StatisticsGen finishes running, we can visualize the outputted statistics. Try playing with the different plots!

In [13]:
# Visualize the output
context.show(statistics_gen.outputs['statistics'])

## Data Statistics with SchemaGen

The SchemaGen component generates a schema based on your data statistics. (A schema defines the expected bounds, types, and properties of the features in your dataset.) It also uses the TensorFlow Data Validation library.

SchemaGen will take as input the statistics that we generated with StatisticsGen, looking at the training split by default.

In [14]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

After SchemaGen finishes running, we can visualize the generated schema as a table. Each feature in your dataset shows up as a row in the schema table, alongside its properties. The schema also captures all the values that a categorical feature takes on, denoted as its domain.

In [15]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'DBP',FLOAT,required,single,-
'EtCO2',FLOAT,required,single,-
'HR',INT,required,single,-
'MAP',FLOAT,required,single,-
'O2Sat',FLOAT,required,single,-
'Resp',INT,required,single,-
'SBP',FLOAT,required,single,-
'Temp',FLOAT,required,single,-
'isSepsis',INT,required,single,-


## Detect anomalies with ExampleValidator

The ExampleValidator component detects anomalies in your data, based on the expectations defined by the schema. It also uses the TensorFlow Data Validation library.

ExampleValidator will take as input the statistics from StatisticsGen, and the schema from SchemaGen.

In [16]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=False)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

After ExampleValidator finishes running, we can visualize the anomalies as a table. In the anomalies table, we can see that there are no anomalies. This is what we'd expect, since this the first dataset that we've analyzed and the schema is tailored to it. You should review this schema -- anything unexpected means an anomaly in the data. Once reviewed, the schema can be used to guard future data, and anomalies produced here can be used to debug model performance, understand how your data evolves over time, and identify data errors.

In [17]:
# visualize the anomalies in a table
context.show(example_validator.outputs['anomalies'])

## Transform

The Transform component performs feature engineering for both training and serving. It uses the TensorFlow Transform library.

Transform will take as input the data from ExampleGen, the schema from SchemaGen, as well as a module that contains user-defined Transform code.

In [43]:
_vitals_constants_module_file = 'vitals_constants.py'

The %%writefile cell magic will save the contents of the cell as a .py file on disk. This allows the Transform component to load your code as a module.

In [51]:
%%writefile {_vitals_constants_module_file}

# dropped MAP need to handle NaN
NUMERICAL_FEATURES = ['HR','Temp','Resp', 'isSepsis']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

# Keys
LABEL_KEY = 'isSepsis'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Overwriting vitals_constants.py


In [45]:
_vitals_transform_module_file = 'vitals_transform.py'

In [46]:
%%writefile {_vitals_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
#if 'google.colab' in sys.modules:  # Testing to see if we're doing development
import importlib
importlib.reload(vitals_constants)

_NUMERICAL_FEATURES = vitals_constants.NUMERICAL_FEATURES
_LABEL_KEY = vitals_constants.LABEL_KEY


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[vitals_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  # Was this patient septic?
  vitals_septic = _fill_in_missing(inputs[_LABEL_KEY])
  """
  outputs[_LABEL_KEY] = tf.where(
      tf.math.is_nan(vitals_septic),
      tf.cast(tf.zeros_like(vitals_septic), tf.int64),
  #    # Test if the patient is septic == 1
      tf.cast(
          tf.greater(vitals_septic, tf.constant(0), tf.int64)
  """
  return outputs

Overwriting vitals_transform.py


In [47]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_vitals_transform_module_file))
context.run(transform, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-prototype/sepsis/notebooks/vitals_transform.py' (including modules: ['vitals_trainer', 'vitals_constants', 'vitals_transform']).
INFO:absl:User module package has hash fingerprint version da5b17a0b1bc1ef5a43470270d623311c9be143aa33d399631975e9ed227b9a2.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpkpubasoh/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpfeem9b5e', '--dist-dir', '/tmp/tmpd9e3fhbp']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying vitals_trainer.py -> build/lib
copying vitals_constants.py -> build/lib
copying vitals_transform.py -> build/lib
installing to /tmp/tmpfeem9b5e
running install
running install_lib


/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


copying build/lib/vitals_trainer.py -> /tmp/tmpfeem9b5e
copying build/lib/vitals_constants.py -> /tmp/tmpfeem9b5e
copying build/lib/vitals_transform.py -> /tmp/tmpfeem9b5e
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt


INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Transform-0.0+da5b17a0b1bc1ef5a43470270d623311c9be143aa33d399631975e9ed227b9a2-py3-none-any.whl'; target user module is 'vitals_transform'.
INFO:absl:Full user module path is 'vitals_transform@/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Transform-0.0+da5b17a0b1bc1ef5a43470270d623311c9be143aa33d399631975e9ed227b9a2-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'vitals_transform@/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Transform-0.0+da5b17a0b1bc1ef5a

...

Processing /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Transform-0.0+da5b17a0b1bc1ef5a43470270d623311c9be143aa33d399631975e9ed227b9a2-py3-none-any.whl


INFO:absl:Successfully installed '/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Transform-0.0+da5b17a0b1bc1ef5a43470270d623311c9be143aa33d399631975e9ed227b9a2-py3-none-any.whl'.
INFO:absl:Feature DBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature EtCO2 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature MAP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature O2Sat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Resp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature SBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature isSepsis has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature patient_id has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature record_date has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature record_ti

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/Transform/transform_graph/10/.temp_path/tftransform_tmp/e99506960fd84c5ab0d6f49a8f107e76/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/Transform/transform_graph/10/.temp_path/tftransform_tmp/d3a040b727c34d158195657beaccc104/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [36]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}),
 'post_transform_schema': OutputChannel(arti

In [37]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

The transformed_metadata subdirectory contains the schema of the preprocessed data. The transform_fn subdirectory contains the actual preprocessing graph. The metadata subdirectory contains the schema of the original data.

We can also take a look at the first three transformed examples:

In [38]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "HR_xf"
    value {
      float_list {
        value: -0.6613534092903137
      }
    }
  }
  feature {
    key: "Resp_xf"
    value {
      float_list {
        value: -0.1939026564359665
      }
    }
  }
  feature {
    key: "Temp_xf"
    value {
      float_list {
        value: 2.2171924114227295
      }
    }
  }
  feature {
    key: "isSepsis_xf"
    value {
      float_list {
        value: -0.27875474095344543
      }
    }
  }
}



The Trainer component will train a model that you define in TensorFlow. Default Trainer support Estimator API, to use Keras API, you need to specify Generic Trainer by setup custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor) in Trainer's contructor.

Trainer takes as input the schema from SchemaGen, the transformed data and graph from Transform, training parameters, as well as a module that contains user-defined model code.

In [39]:
_vitals_trainer_module_file = 'vitals_trainer.py'

In [48]:
%%writefile {_vitals_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
#if 'google.colab' in sys.modules:  # Testing to see if we're doing development
import importlib
importlib.reload(vitals_constants)

_LABEL_KEY = vitals_constants.LABEL_KEY

_BATCH_SIZE = 40

print(_LABEL_KEY)


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY
      ),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying sepsis data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)

  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Overwriting vitals_trainer.py


In [49]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_vitals_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-prototype/sepsis/notebooks/vitals_trainer.py' (including modules: ['vitals_trainer', 'vitals_constants', 'vitals_transform']).
INFO:absl:User module package has hash fingerprint version 8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpn2i824ck/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpjqkkv3f0', '--dist-dir', '/tmp/tmpdj0gikn7']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying vitals_trainer.py -> build/lib
copying vitals_constants.py -> build/lib
copying vitals_transform.py -> build/lib
installing to /tmp/tmpjqkkv3f0
running install
running install_lib


/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


copying build/lib/vitals_trainer.py -> /tmp/tmpjqkkv3f0
copying build/lib/vitals_constants.py -> /tmp/tmpjqkkv3f0
copying build/lib/vitals_transform.py -> /tmp/tmpjqkkv3f0
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt


INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28-py3-none-any.whl'; target user module is 'vitals_trainer'.
INFO:absl:Full user module path is 'vitals_trainer@/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 10000\n}', 'eval_args': '{\n  "num_steps": 5000\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'vitals_tra

writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpjqkkv3f0/tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28-py3.8.egg-info
running install_scripts
creating /tmp/tmpjqkkv3f0/tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28.dist-info/WHEEL
creating '/tmp/tmpdj0gikn7/tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28-py3-none-any.whl' and adding '/tmp/tmpjqkkv3f0' to it
adding 'vitals_constants.py'
adding 'vitals_trainer.py'
adding 'vitals_transform.py'
adding 'tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28.dist-info/METADATA'
adding 'tfx_user_code_Trainer

INFO:absl:Successfully installed '/tmp/tfx-interactive-2022-04-21T17_11_30.851786-c2lbnqna/_wheels/tfx_user_code_Trainer-0.0+8834b7a2309e50a919b78eb4787641b7d8e89725d4f7a5f35c8f5c2e47562b28-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature HR_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Resp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Temp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature isSepsis_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature HR_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Resp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Temp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature isSepsis_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature HR_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Resp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Temp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature isSepsis_xf has a shape . Setting to Dens

isSepsis


ValueError: The `label_key` provided (isSepsis) must be one of the following tensorsnames: dict_keys(['HR_xf', 'Resp_xf', 'Temp_xf', 'isSepsis_xf']).

In [ ]:
# Imported files such as sepsis_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import sepsis_constants
import sys
#if 'google.colab' in sys.modules:  # Testing to see if we're doing development
import importlib
importlib.reload(sepsis_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=sepsis_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
       # tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        #tfma.SlicingSpec(
        #    feature_keys=['trip_start_hour'])
    ])

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

In [ ]:
evaluator.outputs

In [ ]:
context.show(evaluator.outputs['evaluation'])

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)